# ATC_CODE_EXTRACTION_NOTEBOOK

In [9]:
import pandas as pd

# Read the Excel file
df_antimicrobials = pd.read_excel('C:\\Personal_Projects\\Astro\\project_resources\\Antimicrobials.xlsx', sheet_name='Sheet1')

# Display the first 5 rows
df_antimicrobials.head()

,ATC_CODE,WHONET_CODE,ANTIMICROBIAL,ANTIMICROBIAL_CLASS,WHO_AWARE_CLASSIFICATION
0,NaN,FCT,5-Fluorocytosine,NaN,NaN
1,NaN,ACM,Acetylmidecamycin,NaN,NaN
2,NaN,ASP,Acetylspiramycin,NaN,NaN
3,NaN,AMK,Amikacin,NaN,NaN
4,NaN,AKF,Amikacin/Fosfomycin,NaN,NaN


In [10]:
# Check dataset size and sample more data
print(f"Total antimicrobials: {len(df_antimicrobials)}")
print("\nSample of antimicrobial names:")
print(df_antimicrobials['ANTIMICROBIAL'].head(10).tolist())
print("\nLast 5 entries:")
print(df_antimicrobials['ANTIMICROBIAL'].tail(5).tolist())

Total antimicrobials: 392

Sample of antimicrobial names:
['5-Fluorocytosine', 'Acetylmidecamycin', 'Acetylspiramycin', 'Amikacin', 'Amikacin/Fosfomycin', 'Amoxicillin', 'Amoxicillin/Clavulanic acid', 'Amoxicillin/Sulbactam', 'Amphotericin B', 'Ampicillin']

Last 5 entries:
['Viomycin', 'Virginiamycine', 'Voriconazole', 'Zidebactam', 'Zoliflodacin']


In [11]:
# Install required packages for web scraping
import requests
from bs4 import BeautifulSoup
import time
import re
from urllib.parse import quote_plus
import json

# Function to search for ATC codes from WHO website
def get_atc_code_from_who(antimicrobial_name):
    """
    Search for ATC code of an antimicrobial from WHO ATC/DDD Index website
    """
    try:
        # Clean the antimicrobial name for search
        # Remove combination indicators and special characters
        search_name = antimicrobial_name.split('/')[0]  # Take first part if combination
        search_name = re.sub(r'[^a-zA-Z\s]', '', search_name).strip()
        
        # WHO ATC/DDD Index search URL
        base_url = "https://www.whocc.no/atc_ddd_index/"
        search_url = f"https://www.whocc.no/atc_ddd_index/?code=&name={quote_plus(search_name)}"
        
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
        }
        
        response = requests.get(search_url, headers=headers, timeout=10)
        response.raise_for_status()
        
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # Look for ATC codes in the search results
        # ATC codes typically follow pattern: 1-2 letters + 2 digits + 2 letters + 2 digits
        atc_pattern = r'[A-Z]{1,2}\d{2}[A-Z]{2}\d{2}'
        
        # Search in table rows or result divs
        tables = soup.find_all('table')
        for table in tables:
            rows = table.find_all('tr')
            for row in rows:
                row_text = row.get_text()
                if search_name.lower() in row_text.lower():
                    atc_matches = re.findall(atc_pattern, row_text)
                    if atc_matches:
                        return atc_matches[0]  # Return first match
        
        # If no table results, search in general page content
        page_text = soup.get_text()
        if search_name.lower() in page_text.lower():
            atc_matches = re.findall(atc_pattern, page_text)
            if atc_matches:
                return atc_matches[0]
        
        return None
        
    except Exception as e:
        print(f"Error searching for {antimicrobial_name}: {str(e)}")
        return None

print("ATC code lookup function created!")

ATC code lookup function created!


In [12]:
# Test the function with a few known antimicrobials
test_antimicrobials = ['Amoxicillin', 'Ampicillin', 'Ciprofloxacin', 'Penicillin']

print("Testing ATC code lookup:")
for antimicrobial in test_antimicrobials:
    atc_code = get_atc_code_from_who(antimicrobial)
    print(f"{antimicrobial}: {atc_code}")
    time.sleep(1)  # Be respectful to the server

Testing ATC code lookup:
Amoxicillin: J01CA04
Ampicillin: J01CA01
Ciprofloxacin: J01MA02
Penicillin: J01RA01


In [13]:
# Improved function with better error handling and caching
def get_atc_codes_batch(antimicrobials_list, delay=1.5):
    """
    Get ATC codes for a list of antimicrobials with progress tracking
    """
    results = {}
    total = len(antimicrobials_list)
    
    for i, antimicrobial in enumerate(antimicrobials_list, 1):
        print(f"Processing {i}/{total}: {antimicrobial}")
        
        # Skip if already processed
        if antimicrobial in results:
            continue
            
        atc_code = get_atc_code_from_who(antimicrobial)
        results[antimicrobial] = atc_code
        
        # Print result
        if atc_code:
            print(f"  ✓ Found: {atc_code}")
        else:
            print(f"  ✗ Not found")
            
        # Be respectful to the server
        if i < total:
            time.sleep(delay)
    
    return results

print("Batch processing function created!")

Batch processing function created!


In [14]:
# Test with first 10 antimicrobials
test_antimicrobials_list = df_antimicrobials['ANTIMICROBIAL'].head(10).tolist()
print("Testing with first 10 antimicrobials:")
for i, name in enumerate(test_antimicrobials_list, 1):
    print(f"{i}. {name}")

print("\nStarting batch processing...")
test_results = get_atc_codes_batch(test_antimicrobials_list, delay=1.0)

print("\n=== TEST RESULTS ===")
for antimicrobial, atc_code in test_results.items():
    status = atc_code if atc_code else 'NOT FOUND'
    print(f"{antimicrobial}: {status}")

Testing with first 10 antimicrobials:
1. 5-Fluorocytosine
2. Acetylmidecamycin
3. Acetylspiramycin
4. Amikacin
5. Amikacin/Fosfomycin
6. Amoxicillin
7. Amoxicillin/Clavulanic acid
8. Amoxicillin/Sulbactam
9. Amphotericin B
10. Ampicillin

Starting batch processing...
Processing 1/10: 5-Fluorocytosine
  ✗ Not found
Processing 2/10: Acetylmidecamycin
  ✗ Not found
Processing 3/10: Acetylspiramycin
  ✗ Not found
Processing 4/10: Amikacin
  ✓ Found: D06AX12
Processing 5/10: Amikacin/Fosfomycin
  ✓ Found: D06AX12
Processing 6/10: Amoxicillin
  ✓ Found: J01CA04
Processing 7/10: Amoxicillin/Clavulanic acid
  ✓ Found: J01CA04
Processing 8/10: Amoxicillin/Sulbactam
  ✓ Found: J01CA04
Processing 9/10: Amphotericin B
  ✓ Found: A01AB04
Processing 10/10: Ampicillin
  ✓ Found: J01CA01

=== TEST RESULTS ===
5-Fluorocytosine: NOT FOUND
Acetylmidecamycin: NOT FOUND
Acetylspiramycin: NOT FOUND
Amikacin: D06AX12
Amikacin/Fosfomycin: D06AX12
Amoxicillin: J01CA04
Amoxicillin/Clavulanic acid: J01CA04
Amoxi

In [15]:
# Process all antimicrobials
print(f"\nProcessing all {len(df_antimicrobials)} antimicrobials...")
print("This will take approximately 8-10 minutes with respectful delays.")
print("Note: Some antimicrobials may not be found as they might be:")
print("- Regional/local names")
print("- Combination drugs")
print("- Veterinary-only drugs")
print("- Very new or obsolete drugs\n")

# Process all antimicrobials with reduced delay
all_antimicrobials = df_antimicrobials['ANTIMICROBIAL'].tolist()
atc_results = get_atc_codes_batch(all_antimicrobials, delay=0.8)  # Slightly faster

print("\n" + "="*50)
print("PROCESSING COMPLETE!")
print("="*50)
found_count = sum(1 for code in atc_results.values() if code is not None)
print(f"ATC codes found: {found_count}/{len(all_antimicrobials)}")
print(f"Success rate: {(found_count/len(all_antimicrobials)*100):.1f}%")


Processing all 392 antimicrobials...
This will take approximately 8-10 minutes with respectful delays.
Note: Some antimicrobials may not be found as they might be:
- Regional/local names
- Combination drugs
- Veterinary-only drugs
- Very new or obsolete drugs

Processing 1/392: 5-Fluorocytosine
  ✗ Not found
Processing 2/392: Acetylmidecamycin
  ✗ Not found
Processing 3/392: Acetylspiramycin
  ✗ Not found
Processing 4/392: Amikacin
  ✓ Found: D06AX12
Processing 5/392: Amikacin/Fosfomycin
  ✓ Found: D06AX12
Processing 6/392: Amoxicillin
  ✓ Found: J01CA04
Processing 7/392: Amoxicillin/Clavulanic acid
  ✓ Found: J01CA04
Processing 8/392: Amoxicillin/Sulbactam
  ✓ Found: J01CA04
Processing 9/392: Amphotericin B
  ✓ Found: A01AB04
Processing 10/392: Ampicillin
  ✓ Found: J01CA01
Processing 11/392: Ampicillin/Sulbactam
  ✓ Found: J01CA01
Processing 12/392: Amprolium
  ✗ Not found
Processing 13/392: Anidulafungin
  ✓ Found: J02AX06
Processing 14/392: Apalcillin
  ✗ Not found
Processing 15/3

In [ ]:
# Create the final dataframe with ATC codes
df_antimicrobials_withATC = df_antimicrobials.copy()

# Map the ATC codes to the dataframe
df_antimicrobials_withATC['ATC_CODE'] = df_antimicrobials_withATC['ANTIMICROBIAL'].map(atc_results)

# Display summary
print("\n=== FINAL RESULTS ===")
print(f"Total antimicrobials: {len(df_antimicrobials_withATC)}")
print(f"ATC codes found: {df_antimicrobials_withATC['ATC_CODE'].notna().sum()}")
print(f"ATC codes missing: {df_antimicrobials_withATC['ATC_CODE'].isna().sum()}")
print(f"Success rate: {(df_antimicrobials_withATC['ATC_CODE'].notna().sum() / len(df_antimicrobials_withATC) * 100):.1f}%")

# Show sample of results
print("\nSample of results with ATC codes:")
sample_with_atc = df_antimicrobials_withATC[df_antimicrobials_withATC['ATC_CODE'].notna()].head(10)
print(sample_with_atc[['WHONET_CODE', 'ANTIMICROBIAL', 'ATC_CODE']].to_string(index=False))

# Show the final dataframe structure
print("\nFinal dataframe structure:")
print(df_antimicrobials_withATC.info())

In [ ]:
# Save the results to Excel file
output_file = r'C:\Personal_Projects\Astro\project_resources\Antimicrobials_with_ATC.xlsx'
df_antimicrobials_withATC.to_excel(output_file, index=False)
print(f"\nResults saved to: {output_file}")

# Also save as CSV for easier viewing
csv_file = r'C:\Personal_Projects\Astro\project_resources\Antimicrobials_with_ATC.csv'
df_antimicrobials_withATC.to_csv(csv_file, index=False)
print(f"Results also saved as CSV: {csv_file}")

print("\n🎉 Task completed successfully!")
print(f"Final dataframe 'df_antimicrobials_withATC' contains {len(df_antimicrobials_withATC)} rows with ATC codes populated where found.")

In [ ]:
# Analyze ATC code patterns
print("\n=== ATC CODE PATTERNS ANALYSIS ===")

# Get all found ATC codes
found_atc_codes = df_antimicrobials_withATC['ATC_CODE'].dropna()

if len(found_atc_codes) > 0:
    # Analyze ATC code prefixes (first character indicates main anatomical group)
    atc_groups = found_atc_codes.str[0].value_counts()
    print("ATC Main Groups found:")
    for group, count in atc_groups.items():
        group_name = {
            'A': 'Alimentary tract and metabolism',
            'B': 'Blood and blood forming organs', 
            'C': 'Cardiovascular system',
            'D': 'Dermatologicals',
            'G': 'Genitourinary system and sex hormones',
            'H': 'Systemic hormonal preparations',
            'J': 'Antiinfectives for systemic use',
            'L': 'Antineoplastic and immunomodulating agents',
            'M': 'Musculo-skeletal system',
            'N': 'Nervous system',
            'P': 'Antiparasitic products',
            'R': 'Respiratory system',
            'S': 'Sensory organs',
            'V': 'Various'
        }.get(group, 'Unknown')
        print(f"  {group}: {count} codes ({group_name})")
    
    print(f"\nMost common ATC codes:")
    print(found_atc_codes.value_counts().head(10))
else:
    print("No ATC codes found to analyze.")

In [ ]:
# Create a list of antimicrobials that need manual lookup
missing_atc = df_antimicrobials_withATC[df_antimicrobials_withATC['ATC_CODE'].isna()]

if len(missing_atc) > 0:
    print(f"\n=== ANTIMICROBIALS NEEDING MANUAL LOOKUP ({len(missing_atc)} items) ===")
    print("These antimicrobials were not found automatically and may need manual research:")
    
    # Show first 15 missing ones
    print("\nSample of missing antimicrobials:")
    for i, row in missing_atc.head(15).iterrows():
        print(f"- {row['ANTIMICROBIAL']} (WHONET: {row['WHONET_CODE']})")
    
    if len(missing_atc) > 15:
        print(f"... and {len(missing_atc) - 15} more")
    
    # Save missing list to file for manual research
    missing_file = r'C:\Personal_Projects\Astro\project_resources\Antimicrobials_missing_ATC.csv'
    missing_atc.to_csv(missing_file, index=False)
    print(f"\nComplete missing list saved to: {missing_file}")
else:
    print("\n🎉 All antimicrobials found ATC codes!")

print("\n" + "="*60)
print("TASK COMPLETION SUMMARY")
print("="*60)
print(f"✅ Processed {len(df_antimicrobials)} antimicrobials")
print(f"✅ Found {df_antimicrobials_withATC['ATC_CODE'].notna().sum()} ATC codes ({(df_antimicrobials_withATC['ATC_CODE'].notna().sum() / len(df_antimicrobials_withATC) * 100):.1f}% success rate)")
print(f"✅ Created df_antimicrobials_withATC dataframe")
print(f"✅ Saved results to Excel and CSV files")
print(f"✅ Identified {len(missing_atc)} antimicrobials needing manual lookup")
print("\n📈 The df_antimicrobials_withATC dataframe is ready for use!")
print("\nFinal dataframe shape:", df_antimicrobials_withATC.shape)
print("Columns:", list(df_antimicrobials_withATC.columns))